In [113]:
# Evaluating OpenQA Evaluation table1
import pandas as pd

In [114]:
df = pd.read_json('data/TQ-nli-qwen-plus-s42.json')
# df = pd.read_json('data/NQ-nli-gpt35.json')

In [115]:
df[df.isna().any(axis=1)]

,qid,system,question,golden_answer,system_answer,golden_judge,golden_statement,system_statement,a2astar,astar2a


In [116]:
df['asup'] = df['a2astar'].apply(lambda x: 'entailment' in x.lower()).astype(int)
df['ainf'] = df['astar2a'].apply(lambda x: 'entailment' in x.lower()).astype(int)

In [117]:
# Ainf | Asup
df['system_judge'] = (df['asup'] | df['ainf']).astype(int)

# Ainf & Asup
#df['system_judge'] = (df['pred_a2astar'] & df['pred_astar2a']).astype(int)

# Asup - Ainf
#df['system_judge'] = (df['pred_a2astar'] & ~df['pred_astar2a']).astype(int)

#df = df.drop(columns=['pred_a2astar', 'pred_astar2a'])
#df.sort_values(by='qid', ascending=True, inplace=True)
#df.to_json('data/NQ-nli-gpt35.json', orient='records', indent=2)

In [118]:
def fscore(tp,fp,fn):
    prec = tp / (tp + fp)
    rec = tp / (tp + fn)
    f1 = 2 * prec * rec / (prec + rec)
    return prec, rec, f1

def eval_df(df):
    tp = df[(df['golden_judge'] == 1) & (df['system_judge'] == 1)].shape[0]
    fp = df[(df['golden_judge'] == 0) & (df['system_judge'] == 1)].shape[0]
    tn = df[(df['golden_judge'] == 0) & (df['system_judge'] == 0)].shape[0]
    fn = df[(df['golden_judge'] == 1) & (df['system_judge'] == 0)].shape[0]

    pos_prec, pos_rec, pos_f1 = fscore(tp, fp, fn)
    neg_prec, neg_rec, neg_f1 = fscore(tn, fn, fp)

    acc = (tp + tn) / (tp + tn + fp + fn)
    macro_f1 = (pos_f1 + neg_f1) / 2
    print(f'Acc: {round(acc, 3)}, Pos F1: {round(pos_f1, 3)}')
    print(f'Pos Prec: {round(pos_prec, 3)}, Pos Rec: {round(pos_rec, 3)}')

In [119]:
systems = ['fid', 'gpt35', 'chatgpt', 'gpt4', 'newbing']
for sys in systems:
    subdf = df[df['system'] == sys]
    print(sys)
    eval_df(subdf)

fid
Acc: 0.966, Pos F1: 0.979
Pos Prec: 0.97, Pos Rec: 0.989
gpt35
Acc: 0.954, Pos F1: 0.97
Pos Prec: 0.977, Pos Rec: 0.964
chatgpt
Acc: 0.957, Pos F1: 0.975
Pos Prec: 0.977, Pos Rec: 0.972
gpt4
Acc: 0.966, Pos F1: 0.981
Pos Prec: 0.983, Pos Rec: 0.979
newbing
Acc: 0.943, Pos F1: 0.968
Pos Prec: 0.986, Pos Rec: 0.95


In [120]:
# count...
for system in systems:
    print(system)
    #print(df[df['system'] == system]['golden_judge'].value_counts())

    print(df[df['system'] == system]['system_judge'].value_counts()[1] / df[df['system'] == system].shape[0])

fid
0.8307533539731682
gpt35
0.7739938080495357
chatgpt
0.8405572755417957
gpt4
0.8988648090815273
newbing
0.8642930856553147
